In [1]:
import pandas as pd
import glob
from difflib import get_close_matches
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif

fbref_files = sorted(glob.glob("D:/#CSProject/Data/Fbref/*.csv"), reverse=True)  # Folder Location Containing Fbref files 
fbref_df = pd.concat([pd.read_csv(file, encoding='utf-8', encoding_errors='replace') for file in fbref_files], ignore_index=True)

football_data_files = sorted(glob.glob("D:/#CSProject/Data/FD/*.csv"), reverse=True)  # Folder Location Containing Football Data files 
football_data_df = pd.concat([pd.read_csv(file, encoding='utf-8', encoding_errors='replace') for file in football_data_files], ignore_index=True)

In [2]:
fbref_df['Date'] = pd.to_datetime(fbref_df['Date'], errors='coerce')  
football_data_df['Date'] = pd.to_datetime(football_data_df['Date'], format='%d/%m/%Y', errors='coerce')  

fbref_df['Date'] = fbref_df['Date'].dt.strftime('%Y-%m-%d')
football_data_df['Date'] = football_data_df['Date'].dt.strftime('%Y-%m-%d')

football_data_df = football_data_df.iloc[:, :23]

fbref_df = fbref_df.rename(columns={
    'Home xG': 'Home_xG',
    'Away xG': 'Away_xG',
    'Home Team': 'Home_Team',
    'Away Team': 'Away_Team'
})

football_data_df = football_data_df.rename(columns={
    'HomeTeam': 'Home_Team',
    'AwayTeam': 'Away_Team',
    'Div': 'League'
})

In [3]:
name_mapping_fbref = {
    'Spal': 'SPAL',
    'Arminia': 'Arminia Bielefeld',
    'Leeds United': 'Leeds',
    'Köln' : 'FC Koln',
    'Vitória' : 'Vitoria Guimaraes',
    'Paços' : 'Pacos Ferreira',
    'VitÃ³ria SetÃºbal' : 'Vitoria Setubal',
    'B-SAD' : 'Belenenses'
}

# Replace old names with new names in the 'Team' column
fbref_df['Home_Team'] = fbref_df['Home_Team'].replace(name_mapping_fbref)
fbref_df['Away_Team'] = fbref_df['Away_Team'].replace(name_mapping_fbref)

name_mapping_fd = {
    'Spal': 'SPAL',
    'Bielefeld': 'Arminia Bielefeld',
    'Man United': 'Manchester Utd',
    'Man City': 'Manchester City',
    'Guimaraes' : 'Vitoria Guimaraes',
    'Ath Bilbao' : 'Athletic Club',
    'Setubal' : 'Vitoria Setubal',
    'Sp Lisbon' : 'Sporting CP',
    'St. Gilloise' : 'Union SG'
}

football_data_df['Home_Team'] = football_data_df['Home_Team'].replace(name_mapping_fd)
football_data_df['Away_Team'] = football_data_df['Away_Team'].replace(name_mapping_fd)

In [4]:
teams_fbref = set(fbref_df['Home_Team'].unique())
teams_football_data = set(football_data_df['Home_Team'].unique())

mapping = {}
for team in teams_fbref:
    closest_match = get_close_matches(team, teams_football_data, n=1)
    if closest_match:
        mapping[team] = closest_match[0]

# Apply the mapping to df1
fbref_df['Home_Team'] = fbref_df['Home_Team'].replace(mapping)
fbref_df['Away_Team'] = fbref_df['Away_Team'].replace(mapping)
football_data_df['Home_Team'] = football_data_df['Home_Team'].replace(mapping)
football_data_df['Away_Team'] = football_data_df['Away_Team'].replace(mapping)

In [5]:
teams_fbref_conflict = set(fbref_df['Home_Team'].unique())
teams_football_data_conflict = set(football_data_df['Home_Team'].unique())

# Identify team names in df1 that are not in df2
conflicts_fbref = teams_fbref_conflict - teams_football_data_conflict

# Identify team names in df2 that are not in df1
conflicts_football_data = teams_football_data_conflict - teams_fbref_conflict

# Display conflicting names
print("Team names in fbref not in football data:", conflicts_fbref)
print("Team names in football data not in fbref:", conflicts_football_data)

Team names in fbref not in football data: set()
Team names in football data not in fbref: set()


In [6]:
label_encoder = LabelEncoder()
football_data_df['FTR'] = label_encoder.fit_transform(football_data_df['FTR'])

In [7]:
# Calculate the percentage of non-empty values in each column for fbref_df
fbref_non_empty = (fbref_df.select_dtypes(include=["number"]).notnull().sum() / len(fbref_df)) * 100
print("Percentage of non-empty values in fbref_df:")
print(fbref_non_empty)

# Calculate the percentage of non-empty values in each column for football_data_df
football_data_non_empty = (football_data_df.select_dtypes(include=["number"]).notnull().sum() / len(football_data_df)) * 100
print("\nPercentage of non-empty values in football_data_df:")
print(football_data_non_empty)

Percentage of non-empty values in fbref_df:
Home_xG       100.0
Away_xG       100.0
Home Goals    100.0
Away Goals    100.0
dtype: float64

Percentage of non-empty values in football_data_df:
FTHG    100.000000
FTAG    100.000000
FTR     100.000000
HTHG     99.975126
HTAG     99.975126
HS       99.968907
AS       99.968907
HST      99.968907
AST      99.968907
HF       98.476463
AF       98.476463
HC       99.968907
AC       99.968907
HY       99.975126
AY       99.975126
HR       99.975126
AR       99.975126
dtype: float64


In [8]:
football_data_df.select_dtypes(include=["number"]).corr()["FTR"].sort_values(ascending=False)

FTR     1.000000
FTHG    0.634763
HTHG    0.423845
HST     0.395731
HS      0.207799
AR      0.102116
HC      0.030580
AY      0.022406
AF     -0.015825
HF     -0.019891
AC     -0.061246
HY     -0.097026
HR     -0.132303
AS     -0.241826
AST    -0.404299
HTAG   -0.415964
FTAG   -0.638351
Name: FTR, dtype: float64

In [9]:
football_data_df_copy = football_data_df.dropna()
# Select only numeric columns (excluding FTR for features)
X = football_data_df_copy.select_dtypes(include=["number"]).drop(columns=["FTR"])
y = football_data_df_copy["FTR"]
# Compute mutual information scores
mi_scores = mutual_info_classif(X, y)
# Convert to Pandas Series and sort
mi_results = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
print(mi_results)

FTHG    0.313377
FTAG    0.288285
HTHG    0.099554
HST     0.096725
AST     0.095408
HTAG    0.091955
AS      0.034426
HS      0.033108
AR      0.013636
HY      0.012280
HR      0.011809
AC      0.007636
AF      0.004546
HF      0.003344
HC      0.000000
AY      0.000000
dtype: float64


In [10]:
football_data_df = football_data_df[['Date', 'League', 'Home_Team', 'Away_Team', 
                                     "FTR", "FTHG", "FTAG", "HS", "AS", "HST", "AST"]]
# Merge on Date, HomeTeam, and AwayTeam
combined_df = pd.merge(
    football_data_df,
    fbref_df[['Date', 'Home_Team', 'Away_Team', 'Home_xG', 'Away_xG']],  # Only select relevant columns
    on=['Date', 'Home_Team', 'Away_Team'],
    how='left'
)
combined_df = combined_df.dropna()

In [11]:
def compute_gd_elo(df, k=32, initial_rating=1000):
    """
    Compute Goal-Based Elo ratings for each team before each match,
    storing and restoring Elo when a team returns to a league.
    Implements the method by Hvattum & Arntzen (2010).
    """
    teams = {}
    team_leagues = {}
    league_history = {}
    home_elo_list, away_elo_list = [], []

    def goal_multiplier(goal_diff):
        """Adjust K based on goal difference."""
        return 1 + (abs(goal_diff) ** 0.8)  # Scaling function

    for idx, row in df.iterrows():
        home_team, away_team = row['Home_Team'], row['Away_Team']
        home_goals, away_goals = row['FTHG'], row['FTAG']
        goal_diff = home_goals - away_goals  # Goal difference
        home_league, away_league = row['League'], row['League']

        # Initialize Elo if new team
        if home_team not in teams:
            teams[home_team] = initial_rating
        if away_team not in teams:
            teams[away_team] = initial_rating

        home_elo, away_elo = teams[home_team], teams[away_team]

        home_elo_list.append(home_elo)
        away_elo_list.append(away_elo)

        # Expected scores
        expected_home = 1.0 / (1.0 + 10 ** ((away_elo - home_elo) / 400))
        expected_away = 1.0 - expected_home

        # Actual outcome
        if goal_diff > 0:
            score_home, score_away = 1.0, 0.0
        elif goal_diff < 0:
            score_home, score_away = 0.0, 1.0
        else:
            score_home, score_away = 0.5, 0.5

        # K-factor adjustment
        adjusted_k = k * goal_multiplier(goal_diff)

        # Update Elo ratings
        teams[home_team] = home_elo + adjusted_k * (score_home - expected_home)
        teams[away_team] = away_elo + adjusted_k * (score_away - expected_away)

    df['Home_ELO'] = home_elo_list
    df['Away_ELO'] = away_elo_list
    return df

In [12]:
def compute_streaks(df):
    # Initialize dictionary to store streaks for each team
    team_streaks = {}

    # Lists to store streaks for the dataframe
    home_win_streaks, away_win_streaks = [], []
    home_loss_streaks, away_loss_streaks = [], []
    home_unbeaten_streaks, away_unbeaten_streaks = [], []
    home_winless_streaks, away_winless_streaks = [], []

    # Loop through each match in the dataframe
    for idx, row in df.iterrows():
        home_team, away_team = row['Home_Team'], row['Away_Team']
        ftr = row['FTR']  # FTR = 2 (Home Win), 0 (Away Win), 1 (Draw)

        # Initialize streaks if the team is new
        if home_team not in team_streaks:
            team_streaks[home_team] = {
                'home_win': 0, 'away_win': 0, 'home_loss': 0, 'away_loss': 0,
                'home_unbeaten': 0, 'away_unbeaten': 0, 'home_winless': 0, 'away_winless': 0
            }
        if away_team not in team_streaks:
            team_streaks[away_team] = {
                'home_win': 0, 'away_win': 0, 'home_loss': 0, 'away_loss': 0,
                'home_unbeaten': 0, 'away_unbeaten': 0, 'home_winless': 0, 'away_winless': 0
            }

        # Store current streaks before update
        home_win_streaks.append(team_streaks[home_team]['home_win'])
        away_win_streaks.append(team_streaks[away_team]['away_win'])
        home_loss_streaks.append(team_streaks[home_team]['home_loss'])
        away_loss_streaks.append(team_streaks[away_team]['away_loss'])
        home_unbeaten_streaks.append(team_streaks[home_team]['home_unbeaten'])
        away_unbeaten_streaks.append(team_streaks[away_team]['away_unbeaten'])
        home_winless_streaks.append(team_streaks[home_team]['home_winless'])
        away_winless_streaks.append(team_streaks[away_team]['away_winless'])

        # Update streaks based on result
        if ftr == 2:  # Home Win
            team_streaks[home_team]['home_win'] += 1
            team_streaks[away_team]['away_win'] = 0  # Reset away win streak

            team_streaks[home_team]['home_loss'] = 0  # Reset home loss streak
            team_streaks[away_team]['away_loss'] += 1  # Increase away loss streak

            team_streaks[home_team]['home_unbeaten'] += 1  # Increase home unbeaten streak
            team_streaks[away_team]['away_unbeaten'] = 0  # Reset away unbeaten streak

            team_streaks[home_team]['home_winless'] = 0  # Reset home winless streak
            team_streaks[away_team]['away_winless'] += 1  # Increase away winless streak

        elif ftr == 0:  # Away Win
            team_streaks[away_team]['away_win'] += 1
            team_streaks[home_team]['home_win'] = 0  # Reset home win streak

            team_streaks[away_team]['away_loss'] = 0  # Reset away loss streak
            team_streaks[home_team]['home_loss'] += 1  # Increase home loss streak

            team_streaks[away_team]['away_unbeaten'] += 1  # Increase away unbeaten streak
            team_streaks[home_team]['home_unbeaten'] = 0  # Reset home unbeaten streak

            team_streaks[away_team]['away_winless'] = 0  # Reset away winless streak
            team_streaks[home_team]['home_winless'] += 1  # Increase home winless streak

        else:  # Draw
            team_streaks[home_team]['home_win'] = 0  # Reset home win streak
            team_streaks[away_team]['away_win'] = 0  # Reset away win streak

            team_streaks[home_team]['home_loss'] = 0  # Reset home loss streak
            team_streaks[away_team]['away_loss'] = 0  # Reset away loss streak

            # Increase unbeaten streaks for both teams (separately tracked)
            team_streaks[home_team]['home_unbeaten'] += 1
            team_streaks[away_team]['away_unbeaten'] += 1

            # Increase winless streaks for both teams
            team_streaks[home_team]['home_winless'] += 1
            team_streaks[away_team]['away_winless'] += 1

    # Add calculated streaks to the dataframe
    df['Home_Win_Streak'] = home_win_streaks
    df['Away_Win_Streak'] = away_win_streaks
    df['Home_Loss_Streak'] = home_loss_streaks
    df['Away_Loss_Streak'] = away_loss_streaks
    df['Home_Unbeaten_Streak'] = home_unbeaten_streaks
    df['Away_Unbeaten_Streak'] = away_unbeaten_streaks
    df['Home_Winless_Streak'] = home_winless_streaks
    df['Away_Winless_Streak'] = away_winless_streaks

    return df

In [13]:
def add_recent_points(df, window_sizes=[5, 10, 20]):
    """
    Adds rolling points over the last 'n' games for home and away teams.

    Parameters:
    df (pd.DataFrame): The match dataset with teams and results.
    window_sizes (list): The number of past games to consider (default: [5, 10]).

    Returns:
    pd.DataFrame: Updated dataframe with rolling points for each team.
    """
    team_points = {}  # Dictionary to track team results over time

    # Initialize columns for each rolling window
    for window in window_sizes:
        df[f"Home_Points_Last_{window}"] = 0
        df[f"Away_Points_Last_{window}"] = 0

    # Iterate over matches
    for idx, row in df.iterrows():
        home_team, away_team = row["Home_Team"], row["Away_Team"]
        ftr = row["FTR"]  # 2 = Home Win, 1 = Draw, 0 = Away Win

        # Assign points based on result
        home_points, away_points = 0, 0
        if ftr == 2:  # Home win
            home_points, away_points = 3, 0
        elif ftr == 1:  # Draw
            home_points, away_points = 1, 1
        elif ftr == 0:  # Away win
            home_points, away_points = 0, 3

        # Initialize history for teams if not present
        if home_team not in team_points:
            team_points[home_team] = []
        if away_team not in team_points:
            team_points[away_team] = []

        # Store rolling points before updating
        for window in window_sizes:
            df.at[idx, f"Home_Points_Last_{window}"] = sum(team_points[home_team][-window:])
            df.at[idx, f"Away_Points_Last_{window}"] = sum(team_points[away_team][-window:])

        # Append the latest match result to the team's history
        team_points[home_team].append(home_points)
        team_points[away_team].append(away_points)

    return df

In [14]:
def add_net_stats(df):
    stat_pairs = {
        "FTG": ("FTHG", "FTAG"),
        "xG": ("Home_xG", "Away_xG"),
        "S": ("HS", "AS"),
        "ST": ("HST", "AST"),
    }

    # Loop through stat pairs and calculate net stats
    for stat_name, (home_stat, away_stat) in stat_pairs.items():
        df[f"Home_Net_{stat_name}"] = df[home_stat] - df[away_stat]
        df[f"Away_Net_{stat_name}"] = df[away_stat] - df[home_stat]

    return df


In [15]:
def add_conceded_stats(df):
    df["FTHG_Conceded"] = df["FTAG"]
    df["FTAG_Conceded"] = df["FTHG"]

    df["HST_Conceded"] = df["AST"]
    df["AST_Conceded"] = df["HST"]
    
    df["HS_Conceded"] = df["AS"]
    df["AS_Conceded"] = df["HS"]
    
    df["Home_xG_Conceded"] = df["Away_xG"]
    df["Away_xG_Conceded"] = df["Home_xG"]
    
    return df

In [16]:
combined_df = compute_gd_elo(combined_df)
combined_df = compute_streaks(combined_df)
combined_df = add_recent_points(combined_df)

In [17]:
combined_df = add_net_stats(combined_df)
combined_df = add_conceded_stats(combined_df)

In [18]:
correlations = combined_df.select_dtypes(include=["number"]).corr()["FTR"].sort_values(ascending=False)

In [19]:
# Filter correlations that contain either "Streak" or "ELO"
engineered_stats_correlations = correlations[correlations.index.str.contains("Streak|Points")]

# Print the filtered correlations
print(engineered_stats_correlations)

Home_Points_Last_20     0.236652
Home_Points_Last_10     0.220119
Home_Points_Last_5      0.189453
Home_Unbeaten_Streak    0.149765
Home_Win_Streak         0.129634
Away_Winless_Streak     0.101443
Away_Loss_Streak        0.084152
Home_Loss_Streak       -0.084334
Home_Winless_Streak    -0.096310
Away_Win_Streak        -0.128216
Away_Unbeaten_Streak   -0.145424
Away_Points_Last_5     -0.192567
Away_Points_Last_10    -0.227454
Away_Points_Last_20    -0.240074
Name: FTR, dtype: float64


In [20]:
# Filter correlations that contain either "Streak" or "ELO"
engineered_stats_correlations2 = correlations[correlations.index.str.contains("Net|ELO|Conceded")]

# Print the filtered correlations
print(engineered_stats_correlations2)

Home_Net_FTG        0.855471
FTAG_Conceded       0.634708
Home_Net_xG         0.533330
Home_Net_ST         0.525301
Away_xG_Conceded    0.411525
AST_Conceded        0.395896
Home_Net_S          0.276586
Home_ELO            0.263620
AS_Conceded         0.207869
HS_Conceded        -0.241777
Away_ELO           -0.272992
Away_Net_S         -0.276586
HST_Conceded       -0.404207
Home_xG_Conceded   -0.419517
Away_Net_ST        -0.525301
Away_Net_xG        -0.533330
FTHG_Conceded      -0.638297
Away_Net_FTG       -0.855471
Name: FTR, dtype: float64


In [21]:
# Select only numeric columns (excluding FTR for features)
X = combined_df.select_dtypes(include=["number"]).drop(columns=["FTR"])
y = combined_df["FTR"]
# Compute mutual information scores
mi_scores = mutual_info_classif(X, y)
# Convert to Pandas Series and sort
mi_results = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

In [22]:
# Select only metrics that contain "Streak" or "ELO"
engineered_stats_mi = mi_results[mi_results.index.str.contains("Streak|Points")]

# Print the filtered metrics
print(engineered_stats_mi)

Home_Points_Last_20     0.031968
Away_Points_Last_20     0.030872
Away_Points_Last_10     0.026188
Home_Points_Last_10     0.026026
Home_Points_Last_5      0.015632
Away_Win_Streak         0.015200
Away_Points_Last_5      0.014219
Home_Win_Streak         0.012722
Home_Winless_Streak     0.011718
Away_Unbeaten_Streak    0.009193
Home_Unbeaten_Streak    0.009004
Away_Winless_Streak     0.004085
Away_Loss_Streak        0.000710
Home_Loss_Streak        0.000000
dtype: float64


In [23]:
# Select only metrics that contain "Streak" or "ELO"
engineered_stats_mi2 = mi_results[mi_results.index.str.contains("Net|ELO|Conceded")]

# Print the filtered metrics
print(engineered_stats_mi2)

Away_Net_FTG        1.071493
Home_Net_FTG        1.071431
FTAG_Conceded       0.310707
FTHG_Conceded       0.285246
Home_Net_xG         0.174071
Away_Net_xG         0.173568
Away_Net_ST         0.160332
Home_Net_ST         0.158483
HST_Conceded        0.100404
Home_xG_Conceded    0.100221
AST_Conceded        0.099227
Away_xG_Conceded    0.095157
Away_Net_S          0.044976
Home_Net_S          0.042142
Home_ELO            0.037502
Away_ELO            0.035725
HS_Conceded         0.031570
AS_Conceded         0.026359
dtype: float64


In [24]:
combined_df.to_csv("D:/#CSProject/Data/combined_datasetxg.csv", index=False)